In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"

from f1tenth_gym_jax import make
import jax
import jax.numpy as jnp

from f1tenth_gym_jax.envs.utils import Param

In [2]:
num_agents = 10
num_envs = 100

In [3]:
env = make("default_v0", num_agents=num_agents, params=Param(produce_scans=True))

2024-10-29 16:44:34.891542: W external/xla/xla/service/gpu/nvptx_compiler.cc:930] The NVIDIA driver's CUDA version is 12.2 which is older than the PTX compiler version 12.6.77. Because the driver is older than the PTX compiler version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [4]:
rng = jax.random.PRNGKey(0)
rng, _rng = jax.random.split(rng)
reset_rng = jax.random.split(_rng, num_envs)

In [5]:
obsv, env_state = jax.vmap(env.reset)(reset_rng)

/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:6217: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  start = asarray(start, dtype=computation_dtype)
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.py:6218: UserWarning: Explicitly requested dtype float64 requested in asarray is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/jax-ml/jax#current-gotchas for more.
  stop = asarray(stop, dtype=computation_dtype)
/home/billyzheng/f1tenth_gym_jax/venv/lib/python3.10/site-packages/jax/_src/numpy/lax_numpy.

TypeError: true_fun and false_fun output must have same type structure, got PyTreeDef(CustomNode(State[()], [*, *, *, *, *, *, *, None])) and PyTreeDef(CustomNode(State[()], [*, *, *, *, *, *, *, *])).